In [13]:
import pandas as pd
import re

In [2]:
raw_data = '../data/raw/'

In [3]:
sales_df = pd.read_pickle(raw_data + 'sales_df.pkl')
sales_df.head()

,PIN,FOLIO,DOR_CODE,NBHC,S_DATE,VI,QU,REA_CD,S_AMT,SUB,STR,S_TYPE,OR_BK,OR_PG,GRANTOR,GRANTEE,DOC_NUM
0,A-23-33-15-ZZZ-000000-00040.0,0000050000,8600,227002.0,1928-03-01,V,U,38,100.0,ZZZ,233315,QC,00810,0435,,HILLSBOROUGH COUNTY,
1,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2019-09-16,V,U,11,100.0,001,012717,DD,26961,0622,SEVIGNY STEPHEN R TRUSTEE,SEVIGNY PAULINE L LIFE ESTATE,2019404632
2,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2014-04-01,V,U,11,100.0,001,012717,TR,22504,1677,SEVIGNY STEPHEN R TRUSTEE,SEVIGNY STEPHEN R TRUSTEE,2014113139
3,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2006-12-05,V,U,11,100.0,001,012717,WD,17411,0866,SEVIGNY STEPHEN R,SEVIGNY STEPHEN R TRUSTEE,2007055159
4,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,1988-10-01,V,U,38,28000.0,001,012717,WD,05528,1424,,,88224230


In [4]:
print(sales_df.shape)
print(sales_df.info())

(2166282, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2166282 entries, 0 to 2166281
Data columns (total 17 columns):
 #   Column    Dtype  
---  ------    -----  
 0   PIN       object 
 1   FOLIO     object 
 2   DOR_CODE  object 
 3   NBHC      float64
 4   S_DATE    object 
 5   VI        object 
 6   QU        object 
 7   REA_CD    object 
 8   S_AMT     float64
 9   SUB       object 
 10  STR       object 
 11  S_TYPE    object 
 12  OR_BK     object 
 13  OR_PG     object 
 14  GRANTOR   object 
 15  GRANTEE   object 
 16  DOC_NUM   object 
dtypes: float64(2), object(15)
memory usage: 281.0+ MB
None


In [5]:
sales_df[sales_df['DOC_NUM'] == '2018356883'].T

,964451
PIN,U-10-29-20-29R-000001-00008.0
FOLIO,0662680766
DOR_CODE,0100
NBHC,220009.0
S_DATE,2018-08-28
VI,I
QU,Q
REA_CD,02
S_AMT,305000.0
SUB,29R


The PIN number is the parcel identifier and is formatted as follows.

Example PIN: A-16-28-19-3A3-000034-00004.0

A 16 28 19 3A3 000034 00004.0 (Municipality) (Section) (Township) (Range) (Land Type-ID) (Block# ) (Lot#)

The MUNICIPALITY CODE indicates whether the property is located in a city or unincorporated county. The code for the City of Tampa is "A", for the City of Temple Terrace "T" and for the City of Plant City "P". All unincorporated county is coded with "U".

The SECTION-TOWNSHIP-RANGE

The Land Type-ID is to further identify the type of land. All platted Subdivisions and Condominiums have been assigned an ID number. This number may be a combination of numbers and letters, such as "3A3". Not all property in the county is platted. For all unplatted, metes and bounds described lands a Land Type-ID of "ZZZ" has been assigned. This way, one look at the PIN number can tell you whether it is platted or unplatted land.

The BLOCK# is 6 digits and will hold the Block Number in platted Subdivisions or Building Number in a Condominium. For unplatted land having a Land Type-ID of "ZZZ" the BLOCK# will carry a uniquely assigned number.

The LOT# is 6 digits and will hold the Lot Numbers in Platted Subdivisions or Unit Numbers in a Condominium. For all unplatted land having a Land Type-ID of "ZZZ" the LOT# will carry a uniquely assigned number.

In [42]:
 # TODO: Use assert and regex to test format of PIN
pattern = re.compile("^[A-Z]-\d{2}-\d{2}-\d{2}-[A-Z0-9]{3}-[0-9]{6}-[A-Z0-9]{5}\.[0-9]$")
# strTest = 'A-16-28-19-3A3-000034-00004.0'
strTest = 'B-16-28-19-3A3-000034-00004.0'
print(pattern.match(strTest))
bool(pattern.match(strTest))

<re.Match object; span=(0, 29), match='B-16-28-19-3A3-000034-00004.0'>


True

In [48]:
# Validate all PIN's match format (i.e. 'A-16-28-19-3A3-000034-00004.0')
pin = sales_df['PIN'].str.match("^[A-Z]-\d{2}-\d{2}-\d{2}-[A-Z0-9]{3}-[A-Z0-9]{6}-[A-Z0-9]{5}\.[A-Z0-9]$")
assert sales_df[~pin].shape[0] == 0

0          True
1          True
2          True
3          True
4          True
           ... 
2166277    True
2166278    True
2166279    True
2166280    True
2166281    True
Name: PIN, Length: 2166282, dtype: bool

In [50]:
assert sales_df[~pin].shape[0] == 0

In [9]:
sales_df[['MUNICIPALITY_CD', 'SECTION_CD', 'TOWNSHIP_CD', 'RANGE_CD', 'LAND_TYPE_ID', 'BLOCK_NUM', 'LOT_NUM']] = sales_df['PIN'].str.split('-', expand=True)
sales_df.head()

,PIN,FOLIO,DOR_CODE,NBHC,S_DATE,VI,QU,REA_CD,S_AMT,SUB,...,GRANTOR,GRANTEE,DOC_NUM,MUNICIPALITY_CD,SECTION_CD,TOWNSHIP_CD,RANGE_CD,LAND_TYPE_ID,BLOCK_NUM,LOT_NUM
0,A-23-33-15-ZZZ-000000-00040.0,0000050000,8600,227002.0,1928-03-01,V,U,38,100.0,ZZZ,...,,HILLSBOROUGH COUNTY,,A,23,33,15,ZZZ,000000,00040.0
1,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2019-09-16,V,U,11,100.0,001,...,SEVIGNY STEPHEN R TRUSTEE,SEVIGNY PAULINE L LIFE ESTATE,2019404632,U,01,27,17,001,000000,00001.0
2,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2014-04-01,V,U,11,100.0,001,...,SEVIGNY STEPHEN R TRUSTEE,SEVIGNY STEPHEN R TRUSTEE,2014113139,U,01,27,17,001,000000,00001.0
3,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2006-12-05,V,U,11,100.0,001,...,SEVIGNY STEPHEN R,SEVIGNY STEPHEN R TRUSTEE,2007055159,U,01,27,17,001,000000,00001.0
4,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,1988-10-01,V,U,38,28000.0,001,...,,,88224230,U,01,27,17,001,000000,00001.0


There are over 300 Department of Revenue (DOR) codes which represent the type of property.  The types of properties include single family homes, condos, malls, night clubs, vacant land, etc.  The DOR code is indicated in the DOR_C field in the parcel data.  

The residential property types that will be used for this analysis include:
- 0100: Single Family Residential
- 0102: Single family home built around a mobile home
- 0106: Townhouse/Villa
- 0200: Mobile Home
- 0400: Condominium
- 0408: Mobile Home Condominium
- 0800: Multi-Family Residential (Duplex, Triplex, Quadplex, etc.) < 10 units
- 0801: Multi-Family Residential (units individually owned)
- 0802: Multi-Family Residential (units rentals)